# Código para la automatización de la extracción de datos. 
#### Alumna: Lucía Vítores López

## ÍNDICE:
1. [Importaciones](#1)


2. [Desarrollo código](#2)

    2.1. [Definición de funciones](#2.1.)
    
    2.2. [Importación de ficheros Excel](#2.2)
    
    2.3. [Definición de algunas variables](#2.3)
    
    2.4. [Diagnóstico](#2.3.1)
    
    2.5. [NHC](#2.3.2)
    
    2.6. [Biopsia](#2.3.3)
    
    2.7. [Biopsia sólida](#2.3.4)
    
    2.8. [Fecha](#2.3.5)
    
    2.9. [Definimos el resto de las variables](#2.4)
    
    2.10. [Ensayos clínicos y tratamientos disponibles](#2.4.1)
    
    2.11. [Número de chip y de paciente](#2.4.2)
    
    2.12. [Mutaciones y derivados](#2.4.3)
    
    2.13. [Mutaciones totales](#2.4.3.1)
    
    2.14. [Genes patogénicos](#2.4.3.2)
    
    2.15. [Variables de interés](#2.5)
    
    
3. [Creación de DataFrame](#3)

## 1. Importaciones. <a id = "1"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [114]:
import imageio as iio
import fitz 
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tabulate import tabulate
import tabula

## 2. Desarrollo código. <a id = "2"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

## 2.1. Definición de funciones. <a id = "2.1."></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [115]:
# Usamos la biblioteca os para acceder al sistema de archivos y verificar cada archivo de la ruta. 
# Se crea una lista vacía llamada ficheros para almacenar los resultados y se usa la os.listdir(ruta) para obtener la lista de los archivos que se encuentran en esa ruta. 
# Con for leemos cada uno de los archivos pdf y lo añadimos a ficheros. 
# Al finalizar, tenemos una lista con todos los archivos de esa ruta. 
def LeerFicherosPDF(ruta):
    ficheros = []
    contenido = os.listdir(ruta)
    for fichero in contenido:
        if os.path.isfile(os.path.join(ruta, fichero)) and fichero.endswith('.pdf'):
            ficheros.append(fichero)
    return ficheros

In [116]:
def LeerDocumento(nombreFichero):
    with fitz.open(nombreFichero) as doc:
        text=""
        for page in doc:
            text = text + page.get_text()
    return text.split('\n')


In [117]:
def BuscarValor(textoBuscar, lines):
    Encontrados = []

    valores = [1 if re.search(textoBuscar, line) else 0 for line in lines]
    valores = [i for i, s in enumerate(valores) if s==1 in valores]

    posiciones = len(textoBuscar)
    
    for i in valores:
        Encontrados.append(lines[i][lines[i].rfind(textoBuscar)+posiciones:].strip())

    return Encontrados

In [118]:
def GenerarImagen(ruta, fichero):
    i = 0   
    zoom = 4
    mat = fitz.Matrix(zoom, zoom)
    with fitz.open(os.path.join(ruta,fichero)) as doc:
        for page in doc:
            i+=1
            val = f"image_{i+1}.png"
            pix = page.get_pixmap(matrix=mat)
            pix.save(os.path.join(ruta,fichero + "_" + val))
            #plt.figure(figsize=(8,15))
            #img = iio.imread(val)
            #plt.imshow(img, cmap='gray')
            #plt.show()

## 2.2. Importación de ficheros Excel. <a id = "2.2"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [119]:
diagnostico = pd.read_excel("C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/Diagnostico.xlsx")
#print(diagnostico)

#Creo un diccionario con los valores tanto del diagnóstico como del número correspondiente. 
diagnosticos_dic = dict(zip(diagnostico["DIAGNÓSTICO"], diagnostico["NÚMERO DIAGNÓSTICO"]))

for diagnostico in diagnosticos_dic:
    valor = diagnosticos_dic.get(diagnostico)
    print(diagnostico, valor)


Carcinoma del tracto biliar 1.0
Colangiocarcinoma 1.1
Carcinoma de la vesícula biliar 1.2
Cáncer de vejiga 2.0
Carcinoma vesicular uroterial 2.1
Cáncer de mama 3.0
Triple negativo de mama 3.1
Neoplasia del sistema nervioso central 4.0
Glioma 4.1
Cáncer cervical 5.0
Condrosarcoma 6.0
Cáncer colorrectal 7.0
Carcinoma de células escamosas cutáneas 8.0
Carcinoma endometrial 9.0
 Cáncer esofágico 10.0
Cáncer gástrico 11.0
Tumor del estroma gastrointestinal 11.1
Cáncer de cabeza y cuello 12.0
Cáncer de riñón 13.0
Cáncer de hígado 14.0
Carcinoma hepatocelular 14.1
Cáncer de pulmón/pulmonar 15.0
Carcinoma pulmonar no microcítico 15.1
Carcinoma pulmonar microcítico 15.2
Melanoma 16.0
Carcinoma de células Merkel 17.0
Mesotelioma 18.0
Neoplasia mixto neuroendocrino no- neuroendocrino 19.0
Carcinoma neuroendocrino 20.0
Carcinoma neuroendocrino de células pequeñas 20.1
Tumor neuroendocrino 21.0
Osteosarcoma 22.0
Cáncer de ovario 23.0
Cáncer de páncreas 24.0
Paraganglioma 25.0
Cáncer de próstata 26.

In [120]:
genes = pd.read_excel("C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/Genes.xlsx")
#print(genes)

mutaciones = genes["GEN"].unique()
#print(mutaciones)

#Creo un diccionario con los valores del 
mutaciones_dic = dict(zip(genes["GEN"], genes["Número gen"]))
mutaciones_dic

for gen, valor in mutaciones_dic.items():
    print(gen, valor)

ABL1 1
AKT1 2
AKT3 3
ALK 4
AR 5
AXL 6
BRAF 7
CCND1 8
CDK4 9
CDK6 10
CTNNB1 11
DDR2 12
EGFR 13
ERBB2 14
ERBB3 15
ERBB4 16
ERG 17
ESR1 18
ETV1 19
ETV4 20
ETV5 21
FGFR1 22
FGFR2 23
FGFR3 24
FGFR4 25
GNA11 26
GNAQ 27
HRAS 28
IDH1 29
IDH2 30
JAK1 31
JAK2 32
JAK3 33
KIT 34
KRAS 35
MAP2K1 36
MAP2K2 37
MET 38
MTOR 39
MYC 40
MYCN 41
NRAS 42
NTRK1 43
NTRK2 44
NTRK3 45
PDGFRA 46
PIK3CA 47
PPARG 48
RAF1 49
RET 50
ROS1 51
SMO 52


In [121]:
genes = pd.read_excel("C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/Genes.xlsx")
#print(genes)

#Escogemos solo los nombres de los genes (primera columna)
mutacion = genes["GEN"].unique()
a = list(mutacion)
print(mutacion)
print(a)
type(a)


['ABL1' 'AKT1' 'AKT3' 'ALK' 'AR' 'AXL' 'BRAF' 'CCND1' 'CDK4' 'CDK6'
 'CTNNB1' 'DDR2' 'EGFR' 'ERBB2' 'ERBB3' 'ERBB4' 'ERG' 'ESR1' 'ETV1' 'ETV4'
 'ETV5' 'FGFR1' 'FGFR2' 'FGFR3' 'FGFR4' 'GNA11' 'GNAQ' 'HRAS' 'IDH1'
 'IDH2' 'JAK1' 'JAK2' 'JAK3' 'KIT' 'KRAS' 'MAP2K1' 'MAP2K2' 'MET' 'MTOR'
 'MYC' 'MYCN' 'NRAS' 'NTRK1' 'NTRK2' 'NTRK3' 'PDGFRA' 'PIK3CA' 'PPARG'
 'RAF1' 'RET' 'ROS1' 'SMO']
['ABL1', 'AKT1', 'AKT3', 'ALK', 'AR', 'AXL', 'BRAF', 'CCND1', 'CDK4', 'CDK6', 'CTNNB1', 'DDR2', 'EGFR', 'ERBB2', 'ERBB3', 'ERBB4', 'ERG', 'ESR1', 'ETV1', 'ETV4', 'ETV5', 'FGFR1', 'FGFR2', 'FGFR3', 'FGFR4', 'GNA11', 'GNAQ', 'HRAS', 'IDH1', 'IDH2', 'JAK1', 'JAK2', 'JAK3', 'KIT', 'KRAS', 'MAP2K1', 'MAP2K2', 'MET', 'MTOR', 'MYC', 'MYCN', 'NRAS', 'NTRK1', 'NTRK2', 'NTRK3', 'PDGFRA', 'PIK3CA', 'PPARG', 'RAF1', 'RET', 'ROS1', 'SMO']


list

## 2.3. Definición de algunas variables. <a id = "2.3"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [122]:
# Estas variables están formadas por la palabra de interés seguido de :, por lo que usamos esos puntos como referencia para usar el resultado.
fecha_Data = []
NHC_Data = [] 
Nbiopsia_Data = [] 
texto_Data = []

Ruta = 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/RE_PDF_inventados'
#RutaPatricia = ... carpeta que nos hemos inventado para trabajar cuando se lo pase

ficheros = LeerFicherosPDF(Ruta)

for ficheroPDF in ficheros:
    #print(os.path.join(Ruta,ficheroPDF))
    lines = LeerDocumento(os.path.join(Ruta,ficheroPDF))

    #print(lines)

    NHC_Data.append(BuscarValor("NHC:", lines))
    Nbiopsia_Data.append(BuscarValor("biopsia:", lines))
    fecha_Data.append(BuscarValor("Fecha:", lines))
    texto_Data.append(BuscarValor("de la muestra:", lines))
   

    GenerarImagen(Ruta,ficheroPDF)

print(NHC_Data)
print(Nbiopsia_Data)
print(fecha_Data)
print(texto_Data)

[['00021', '00021', '00021'], ['00022', '00022', '00022'], ['00023', '00023'], ['00024', '00024'], ['00025', '00025', '00025'], ['00026', '00026', '00026'], ['00027', '00027'], ['00028', '00028']]
[['23B00021-A1', '23B00021-A1', '23B00021-A1'], ['22B00022-A2', '22B00022-A2', '22B00022-A2'], ['21B00023-A1', '21B00023-A1'], ['23B00024-A1', '23B00024-A1'], ['23C00025-A1', '23C00025-A1', '23C00025-A1'], ['23P00026', '23P00026', '23P00026'], ['22B00027-C4', '22B00027-C4'], ['23B00028-A2', '23B00028-A2']]
[['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023']]
[['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar 

## 2.3.1. Diagnóstico. <a id = "2.3.1"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [123]:
textoDiag = []
numeroDiag = []

for i in texto_Data:
    sinduplicados = list(set(i))
    textoDiag.append([x for x in i if x in sinduplicados][0])
print(textoDiag)
    
for diagnostico in textoDiag:
    valor = diagnosticos_dic.get(diagnostico)
    numeroDiag.append(valor)
    print(numeroDiag)
    

['Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Cáncer tiroideo', 'Cáncer gástrico']
[15.1]
[15.1, 15.1]
[15.1, 15.1, 15.1]
[15.1, 15.1, 15.1, 15.1]
[15.1, 15.1, 15.1, 15.1, 15.1]
[15.1, 15.1, 15.1, 15.1, 15.1, 15.1]
[15.1, 15.1, 15.1, 15.1, 15.1, 15.1, 30.0]
[15.1, 15.1, 15.1, 15.1, 15.1, 15.1, 30.0, 11.0]


## 2.3.2 NHC. <a id = "2.3.2"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [124]:
NHC_Data_val = list(set([valor for sublist in NHC_Data for valor in sublist]))
print(NHC_Data_val)
NHC_final = sorted(NHC_Data_val)
print(NHC_final)

['00028', '00023', '00025', '00027', '00021', '00026', '00022', '00024']
['00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028']


## 2.3.3 Biopsia. <a id = "2.3.3"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [125]:
lista_resultante =  []
elementos_vistos = set()

for sublist in Nbiopsia_Data:
    sublist_sin_duplicados = []
    for elemento in sublist:
        if elemento not in elementos_vistos:
            sublist_sin_duplicados.append(elemento)
            elementos_vistos.add(elemento)
    lista_resultante.append(sublist_sin_duplicados)

print(lista_resultante)

NB_values = [elemento for sublist in lista_resultante for elemento in sublist]
print(NB_values)

[['23B00021-A1'], ['22B00022-A2'], ['21B00023-A1'], ['23B00024-A1'], ['23C00025-A1'], ['23P00026'], ['22B00027-C4'], ['23B00028-A2']]
['23B00021-A1', '22B00022-A2', '21B00023-A1', '23B00024-A1', '23C00025-A1', '23P00026', '22B00027-C4', '23B00028-A2']


In [126]:
biopsia = [x[2] for x in NB_values]
biopsia

['B', 'B', 'B', 'B', 'C', 'P', 'B', 'B']

## 2.3.4 Biopsia sólida. <a id = "2.3.4"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [127]:
#B de biopsia = 1
#C de citología = 3
#P de punción = 2
B = "1"
C = "3"
P = "2"
Biopsia_solida = []

for i in biopsia:
    if i == "B":
        Biopsia_solida.append(B)
        print("1")
    elif i == "P":
        Biopsia_solida.append(P)
        print("2")
    else:
        Biopsia_solida.append(C)
        print("3")
        
print(Biopsia_solida)

1
1
1
1
3
2
1
1
['1', '1', '1', '1', '3', '2', '1', '1']


## 2.3.5.Fecha. <a id = "2.3.5"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [128]:
fechas = []
for i in fecha_Data:
    sinduplicados = list(set(i))
    fechas.append([x for x in i if x in sinduplicados][0])

print(fechas)


['24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023']


## 2.4. Definimos el resto de las variables. <a id = "2.4"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

## 2.4.1. Ensayos clínicos y tratamientos disponibles. <a id = "2.4.1"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [129]:
patron = r"(\d+)\s* Ensayos clínicos"
Ruta = 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/RE_PDF_inventados'
ficheros = LeerFicherosPDF(Ruta)

lista_ensayos = []
ensayos_finales = []

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta,ficheroPDF))
    
    ensayos = 0
    for line in lines:
        resultado = re.search(patron, line)
        if resultado:
            ensayos = int(resultado.group(1))
            
    lista_ensayos.append(ensayos)   
    
    print(ficheroPDF + " -> Ensayos: " + str(ensayos))
print(lista_ensayos)


for i in lista_ensayos:
    if i == 0:
        ensayos_finales.append(0)
    else:
        ensayos_finales.append(1)
        
print(ensayos_finales)


Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf -> Ensayos: 6
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf -> Ensayos: 5
Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf -> Ensayos: 0
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf -> Ensayos: 0
Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf -> Ensayos: 3
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf -> Ensayos: 5
Sample_7_v100_869cc724-f5f4-4d61-a708-c818a6c062ef_2023-03-02_11-50-41-677_All.pdf -> Ensayos: 1
Sample_8_v100_14451fbc-44ce-480e-9107-c6ab537a1fda_2023-03-02_11-49-41-641_All.pdf -> Ensayos: 0
[6, 5, 0, 0, 3, 5, 1, 0]
[1, 1, 0, 0, 1, 1, 1, 0]


In [130]:
patron2 = r"(\d+)\s* Tratamientos disponibles"
Ruta = 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/RE_PDF_inventados'
ficheros = LeerFicherosPDF(Ruta)

lista_tratamientos = []
tratamientos_finales = []

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta, ficheroPDF))
    tratamientos = 0
    
    for line in lines:
        resultado = re.search(patron2, line)
        if resultado:
            tratamientos = int(resultado.group(1))
    
    lista_tratamientos.append(tratamientos)
    print(ficheroPDF + " -> Tratamientos: " + str(tratamientos))
    
print(lista_tratamientos)


for i in lista_tratamientos:
    if i == 0:
        tratamientos_finales.append(0)
    else:
        tratamientos_finales.append(1)
print(tratamientos_finales)

Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf -> Tratamientos: 6
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf -> Tratamientos: 1
Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf -> Tratamientos: 0
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf -> Tratamientos: 0
Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf -> Tratamientos: 3
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf -> Tratamientos: 0
Sample_7_v100_869cc724-f5f4-4d61-a708-c818a6c062ef_2023-03-02_11-50-41-677_All.pdf -> Tratamientos: 0
Sample_8_v100_14451fbc-44ce-480e-9107-c6ab537a1fda_2023-03-02_11-49-41-641_All.pdf -> Tratamientos: 1
[6, 1, 0, 0, 3, 0, 0, 1]
[1, 1, 0, 0, 1, 0, 0, 1]


## 2.4.2. Número de chip y de paciente. <a id = "2.4.2"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [131]:
#Leemos todos los nombres de los archivos PDF
Ruta = 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/RE_PDF_inventados'
archivos = os.listdir(Ruta)

for archivo in archivos:
    if os.path.isfile(os.path.join(Ruta, archivo)) and archivo.endswith(".pdf"):
        print(archivo)

Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf
Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf
Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf
Sample_7_v100_869cc724-f5f4-4d61-a708-c818a6c062ef_2023-03-02_11-50-41-677_All.pdf
Sample_8_v100_14451fbc-44ce-480e-9107-c6ab537a1fda_2023-03-02_11-49-41-641_All.pdf


In [132]:
#Sacamos el tercer valor de cada nombre, que es el que indica el número del paciente
Ruta = 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/RE_PDF_inventados'
archivos = os.listdir(Ruta)
numero_paciente = []

for archivo in archivos:
    if os.path.isfile(os.path.join(Ruta, archivo)) and archivo.endswith(".pdf"):
        paciente = os.path.splitext(archivo)[0]
        pacientes = paciente[7]
        numero_paciente.append(pacientes)
        print(numero_paciente)

['1']
['1', '2']
['1', '2', '3']
['1', '2', '3', '4']
['1', '2', '3', '4', '5']
['1', '2', '3', '4', '5', '6']
['1', '2', '3', '4', '5', '6', '7']
['1', '2', '3', '4', '5', '6', '7', '8']


In [133]:
Ruta = 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/RE_PDF_inventados'
archivos = os.listdir(Ruta)
chip2 = []

for archivo in archivos:
    if os.path.isfile(os.path.join(Ruta, archivo)) and archivo.endswith(".pdf"):
        patron = r"v(\d+)_"
        resultado = re.search(patron, archivo)
        if resultado:
            numero_chip = resultado.group(1)
            chip2.append(numero_chip)
print(chip2)

['100', '100', '100', '100', '100', '100', '100', '100']


## 2.4.3. Mutaciones y derivados. <a id = "2.4.3"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

## 2.4.3.1. Mutaciones totales. <a id = "2.4.3.1"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [134]:
#OBTENCIÓN GENES, NÚMERO Y VALORES BIEN!!
ficheros = LeerFicherosPDF(Ruta)
max_mut = 0
genes_mut2 = {}
patron_frecuencia = re.compile(r"\d{2}\.\d{2}")
frecuencias_totales = []

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta,ficheroPDF))
    total_mut = 0
    encontrados2 = []
    lista_frec = []
    
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)
            
            #print(ficheroPDF + " - Existe: " + mutacion)
            #print(posiciones)
            if mutacion == "FGFR4": #solo el tipo p.(P136L)
                if (posicion < len(lines)-1 and lines[posicion + 1] == "p.(P136L)"):
                    pass
                else:
                    total_mut += 1
                    encontrados2.append(mutacion)
                    print(ficheroPDF + " - Existe: " + mutacion)
            else:
                benigno = False
                #print(mutacion)
                for a in range(posicion+1, posicion+10):
                    #print(str(a) + " > " + lines[a])
                    if "Benign" in lines[a]:
                        #print(lines[a])
                        benigno = True
                if benigno == False:
                    total_mut += 1
                    encontrados2.append(mutacion)
                    print(ficheroPDF + " - Existe: " + mutacion)
                if not benigno:
                    for i in lines[posicion:posicion+10]:
                        resultado = re.search(patron_frecuencia, i)
                        if resultado:
                            frec = resultado.group()
                            lista_frec.append(frec)
                            
                    
    
    genes_mut2[ficheroPDF] = encontrados2
    if total_mut > max_mut:
        max_mut = total_mut
        
    frecuencias_totales.append(lista_frec)
    
    print(f"{ficheroPDF} - Número total de mutaciones : {total_mut}")
    print(f"{ficheroPDF} - Frecuencias: {lista_frec}")

    
#print(frecuencias_totales)
#print(genes_mut2)

Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Existe: MYC
Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Número total de mutaciones : 1
Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Frecuencias: ['49.56']
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: KRAS
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Número total de mutaciones : 1
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Frecuencias: ['66.50']
Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf - Número total de mutaciones : 0
Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf - Frecuencias: []
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Existe: FGFR1
Sample_4_v100_03eb528b-ded2-4db9-81fe-e15

In [135]:
frecuencias_totales

[['49.56'],
 ['66.50'],
 [],
 [],
 ['21.75', '21.75'],
 ['27.51', '35.40'],
 ['46.17'],
 ['53.12', '60.64']]

In [136]:
mut = list(genes_mut2.values())
mut

[['MYC'],
 ['KRAS'],
 [],
 ['FGFR1', 'PIK3CA'],
 ['CDK6', 'MET'],
 ['CTNNB1', 'EGFR'],
 ['HRAS'],
 ['KIT', 'MYC']]

In [137]:
num_mutaciones = [len(i) for i in mut]
print(num_mutaciones)

[1, 1, 0, 2, 2, 2, 1, 2]


In [138]:
numero_iden = []
for i in mut:
    valores = [mutaciones_dic.get(gen, 0) for gen in i]
    numero_iden.append(valores)

print(numero_iden)

[[40], [35], [], [22, 47], [10, 38], [11, 13], [28], [34, 40]]


## 2.4.3.2. Genes patogénicos. <a id = "2.4.3.2"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [174]:
#Solo genes patogénicos 
for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta, ficheroPDF))
    
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)

            for a in range(posicion+1, min(posicion+10, len(lines))):
                if "Pathogeni" in lines[a]:
                    print(ficheroPDF + " - Existe: " + mutacion + " - Pathogenic")
                

Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: KRAS - Pathogenic
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf - Existe: CTNNB1 - Pathogenic


In [180]:
patron_frecuencia = re.compile(r"\d{2}\.\d{2}")
frecuenciasPato = []

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta, ficheroPDF))
    lista_frec = []
    
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)
            
            for a in range(posicion+1, min(posicion+10, len(lines))):
                if "Pathogeni" in lines[a]:
                    print(ficheroPDF + "- Existe: " + mutacion + "- Pathogenic")
            
                    for i in lines[posicion:posicion+10]:
                        resultado = re.search(patron_frecuencia, i)
                        if resultado:
                            frec = resultado.group()
                            lista_frec.append(frec)
                            #frecuencias.append(frec)
    frecuenciasPato.append(lista_frec)
                            #print(f"   Frecuencia: {frec}")
        
    print(f"{ficheroPDF} - Frecuencias: {lista_frec}")
    
print(frecuenciasPato)


Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Frecuencias: []
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf- Existe: KRAS- Pathogenic
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Frecuencias: ['66.50']
Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf - Frecuencias: []
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Frecuencias: []
Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf - Frecuencias: []
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf- Existe: CTNNB1- Pathogenic
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf - Frecuencias: ['27.51']
Sample_7_v100_869cc724-f5f4-4d61-a708-c818a6c062ef_2023-03-02_11-50-41-677_All.pdf - Frecuencias: []
Sample_8_v100_14451fbc-44ce-480e-9107-c6ab537a1fda_2023-03-

In [181]:
patogen = {}

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta,ficheroPDF))
    genpato2 = []
    
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)
            
            for a in range(posicion+1, min(posicion+10, len(lines))):
                if "Pathogeni" in lines[a]:
                    genpato2.append(mutacion)
                    print(ficheroPDF + " - Existe: " + mutacion + " - Pathogenic")
            
                    
    
    patogen[ficheroPDF] = genpato2
    
print(patogen)

Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: KRAS - Pathogenic
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf - Existe: CTNNB1 - Pathogenic
{'Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf': [], 'Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf': ['KRAS'], 'Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf': [], 'Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf': [], 'Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf': [], 'Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf': ['CTNNB1'], 'Sample_7_v100_869cc724-f5f4-4d61-a708-c818a6c062ef_2023-03-02_11-50-41-677_All.pdf': [], 'Sample_8_v100_14451fbc-44ce-480e-9107-c6ab537a1fda_2023-03-02_11-49-41-641_All.pdf': []}


In [182]:
patologicos = list(patogen.values())
patologicos

[[], ['KRAS'], [], [], [], ['CTNNB1'], [], []]

In [183]:
numero_iden_pato = []
for i in patologicos:
    valores = [mutaciones_dic.get(gen, 0) for gen in i]
    numero_iden_pato.append(valores)

print(numero_iden_pato)

[[], [35], [], [], [], [11], [], []]


In [184]:
num_mutacionesPato = [len(i) for i in numero_iden_pato]
print(num_mutacionesPato)

[0, 1, 0, 0, 0, 1, 0, 0]


In [ ]:
num_mutacionesPato = [len(i) for i in mut]
print(num_mutaciones)

## ID VARIANTE

In [171]:
patron_genes = re.compile(r"\b(" + "|".join(genes) + r")\S*\b", re.IGNORECASE)
todos = []

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta,ficheroPDF))
    lista_total = []

    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)
            resultado = re.search(patron_genes, i)
            
            if mutacion == "FGFR4":
                if (posicion < len(lines)-1 and lines[posicion + 1] == "p.(P136L)"):
                    pass
                else:
                    print(ficheroPDF + " - Existe: " + mutacion)
            else:
                benigno = False
                for a in range(posicion+1, posicion+10):
                    if "Benign" in lines[a]:
                        benigno = True
                if benigno == False:
                    print(ficheroPDF + " - Existe: " + mutacion)
                    
                for i in lines[posicion:posicion+10]:
                    resultado = re.search(patron_genes, line)
                    if resultado:
                        palabra = resultado.group()
                        lista_total.append(palabra)
    todos.append(lista_total)
                    
    # Agregar la lista total de genes del documento a la lista de todos los documentos
print(todos)

TypeError: expected string or bytes-like object

## 2.5. Variables de interés. <a id = "2.5"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [186]:
# Número de chip 
chip2

['100', '100', '100', '100', '100', '100', '100', '100']

In [188]:
# Número de paciente 
numero_paciente

['1', '2', '3', '4', '5', '6', '7', '8']

In [189]:
# NHC
NHC_final

['00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028']

In [190]:
# Número de biopsia 
NB_values

['23B00021-A1',
 '22B00022-A2',
 '21B00023-A1',
 '23B00024-A1',
 '23C00025-A1',
 '23P00026',
 '22B00027-C4',
 '23B00028-A2']

In [191]:
# Biopsia sólida 
Biopsia_solida

['1', '1', '1', '1', '3', '2', '1', '1']

In [192]:
# Fecha de informe
print(fecha_Data)
print("---------------------------------------------------------------------------------------------------------------------")
print(fechas)

[['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023']]
---------------------------------------------------------------------------------------------------------------------
['24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023']


In [193]:
# Diagnóstico 
print(texto_Data)
print("------------------------------------------------------------------------------------------------------------------")
print(textoDiag)

[['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Cáncer tiroideo'], ['Cáncer gástrico']]
------------------------------------------------------------------------------------------------------------------
['Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Cáncer tiroideo', 'Cáncer gástrico']


In [194]:
# Número de diagnóstico 
numeroDiag

[15.1, 15.1, 15.1, 15.1, 15.1, 15.1, 30.0, 11.0]

In [196]:
# Total del número de mutaciones
num_mutaciones

[1, 1, 0, 2, 2, 2, 1, 2]

In [197]:
# Mutaciones detectadas totales 
mut

[['MYC'],
 ['KRAS'],
 [],
 ['FGFR1', 'PIK3CA'],
 ['CDK6', 'MET'],
 ['CTNNB1', 'EGFR'],
 ['HRAS'],
 ['KIT', 'MYC']]

In [198]:
# % de frecuencia alélica
frecuencias_totales

[['49.56'],
 ['66.50'],
 [],
 [],
 ['21.75', '21.75'],
 ['27.51', '35.40'],
 ['46.17'],
 ['53.12', '60.64']]

In [199]:
# % frecuencias alélicas patológicas 
frecuenciasPato

[[], ['66.50'], [], [], [], ['27.51'], [], []]

In [62]:
# Número de lecturas (% de frecuencia alélica en ARN) 


In [ ]:
# Número de copias (% de frecuencia alélica en copias)


In [201]:
# Nº de la mutación específica detectada 
numero_iden

[[40], [35], [], [22, 47], [10, 38], [11, 13], [28], [34, 40]]

In [202]:
# Ensayos clínicos 
print(lista_ensayos)
print("-----------------------------------------------------------------------------------------------------------------")
print(ensayos_finales)

[6, 5, 0, 0, 3, 5, 1, 0]
-----------------------------------------------------------------------------------------------------------------
[1, 1, 0, 0, 1, 1, 1, 0]


In [203]:
# Fármaco aprobado 
print(lista_tratamientos)
print("-----------------------------------------------------------------------------------------------------------------")
print(tratamientos_finales)

[6, 1, 0, 0, 3, 0, 0, 1]
-----------------------------------------------------------------------------------------------------------------
[1, 1, 0, 0, 1, 0, 0, 1]


## 3. Creación de DataFrame. <a id = "3"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

In [204]:
tablaIdealFinal = pd.DataFrame({'Número de chip': chip2, 'Núme de paciente': numero_paciente, 'NHC': NHC_final, 
                           'Número de biopsia': NB_values, 'Biopsia sólida': Biopsia_solida, 'Fecha de informe': fechas, 
                           'Diagnóstico': textoDiag, 'Número del diagnóstico': numeroDiag, 'Ensayos clínicos': lista_ensayos,
                           'SI/NO ensayo':ensayos_finales, 'Fármaco aprobado': lista_tratamientos, 'SI/NO fármacos': tratamientos_finales, 
                           'Mutaciones detectadas':mut, '% de frecuencia alélica' :frecuencias_totales, 
                            'Número de la mutación específica':numero_iden, 'Total del número de mutaciones': num_mutaciones})

In [205]:
tablaIdealFinal

,Número de chip,Núme de paciente,NHC,Número de biopsia,Biopsia sólida,Fecha de informe,Diagnóstico,Número del diagnóstico,Ensayos clínicos,SI/NO ensayo,Fármaco aprobado,SI/NO fármacos,Mutaciones detectadas,% de frecuencia alélica,Número de la mutación específica,Total del número de mutaciones
0,100,1,00021,23B00021-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,6,1,6,1,[MYC],[49.56],[40],1
1,100,2,00022,22B00022-A2,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,5,1,1,1,[KRAS],[66.50],[35],1
2,100,3,00023,21B00023-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,0,0,0,0,[],[],[],0
3,100,4,00024,23B00024-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,0,0,0,0,"[FGFR1, PIK3CA]",[],"[22, 47]",2
4,100,5,00025,23C00025-A1,3,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,3,1,3,1,"[CDK6, MET]","[21.75, 21.75]","[10, 38]",2
5,100,6,00026,23P00026,2,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,5,1,0,0,"[CTNNB1, EGFR]","[27.51, 35.40]","[11, 13]",2
6,100,7,00027,22B00027-C4,1,24-mar-2023,Cáncer tiroideo,30.0,1,1,0,0,[HRAS],[46.17],[28],1
7,100,8,00028,23B00028-A2,1,24-mar-2023,Cáncer gástrico,11.0,0,0,1,1,"[KIT, MYC]","[53.12, 60.64]","[34, 40]",2


In [206]:
tablaIdealPato = pd.DataFrame({'Número de chip': chip2, 'Núme de paciente': numero_paciente, 'NHC': NHC_final, 
                           'Número de biopsia': NB_values, 'Biopsia sólida': Biopsia_solida, 'Fecha de informe': fechas, 
                           'Diagnóstico': textoDiag, 'Número del diagnóstico': numeroDiag, 'Ensayos clínicos': lista_ensayos,
                           'SI/NO ensayo':ensayos_finales, 'Fármaco aprobado': lista_tratamientos, 'SI/NO fármacos': tratamientos_finales, 
                           'Genes patológicos': patologicos, '% frecuencia alélica':frecuenciasPato,
                            'Número de la mutación específica':numero_iden_pato, 'Total número mutaciones patológicas' :num_mutacionesPato})

In [207]:
tablaIdealPato

,Número de chip,Núme de paciente,NHC,Número de biopsia,Biopsia sólida,Fecha de informe,Diagnóstico,Número del diagnóstico,Ensayos clínicos,SI/NO ensayo,Fármaco aprobado,SI/NO fármacos,Genes patológicos,% frecuencia alélica,Número de la mutación específica,Total número mutaciones patológicas
0,100,1,00021,23B00021-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,6,1,6,1,[],[],[],0
1,100,2,00022,22B00022-A2,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,5,1,1,1,[KRAS],[66.50],[35],1
2,100,3,00023,21B00023-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,0,0,0,0,[],[],[],0
3,100,4,00024,23B00024-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,0,0,0,0,[],[],[],0
4,100,5,00025,23C00025-A1,3,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,3,1,3,1,[],[],[],0
5,100,6,00026,23P00026,2,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,5,1,0,0,[CTNNB1],[27.51],[11],1
6,100,7,00027,22B00027-C4,1,24-mar-2023,Cáncer tiroideo,30.0,1,1,0,0,[],[],[],0
7,100,8,00028,23B00028-A2,1,24-mar-2023,Cáncer gástrico,11.0,0,0,1,1,[],[],[],0


In [219]:
Tabla1Pacientes = pd.DataFrame({'Número de chip': chip2, 'Número de paciente': numero_paciente, 'NHC': NHC_final, 
                                'Número de biopsia': NB_values, 'Biopsia sólida': Biopsia_solida, 'Fecha de informe': fechas})

In [220]:
Tabla1Pacientes

,Número de chip,Número de paciente,NHC,Número de biopsia,Biopsia sólida,Fecha de informe
0,100,1,00021,23B00021-A1,1,24-mar-2023
1,100,2,00022,22B00022-A2,1,24-mar-2023
2,100,3,00023,21B00023-A1,1,24-mar-2023
3,100,4,00024,23B00024-A1,1,24-mar-2023
4,100,5,00025,23C00025-A1,3,24-mar-2023
5,100,6,00026,23P00026,2,24-mar-2023
6,100,7,00027,22B00027-C4,1,24-mar-2023
7,100,8,00028,23B00028-A2,1,24-mar-2023


In [221]:
Tabla2Diagnostico = pd.DataFrame({'Diagnóstico': textoDiag, 'Número del diagnóstico': numeroDiag})

In [222]:
Tabla2Diagnostico

,Diagnóstico,Número del diagnóstico
0,Carcinoma pulmonar no microcítico,15.1
1,Carcinoma pulmonar no microcítico,15.1
2,Carcinoma pulmonar no microcítico,15.1
3,Carcinoma pulmonar no microcítico,15.1
4,Carcinoma pulmonar no microcítico,15.1
5,Carcinoma pulmonar no microcítico,15.1
6,Cáncer tiroideo,30.0
7,Cáncer gástrico,11.0


In [212]:
Tabla3MutacionesEspecificas = pd.DataFrame({'Mutaciones detectadas':mut, 'Número de la mutación específica':numero_iden, 'Total del número de mutaciones': num_mutaciones, 'Porcentaje de frecuencia alélica (ADN)': frecuencias_totales })

In [213]:
Tabla3MutacionesEspecificas

,Mutaciones detectadas,Número de la mutación específica,Total del número de mutaciones,Porcentaje de frecuencia alélica (ADN)
0,[MYC],[40],1,[49.56]
1,[KRAS],[35],1,[66.50]
2,[],[],0,[]
3,"[FGFR1, PIK3CA]","[22, 47]",2,[]
4,"[CDK6, MET]","[10, 38]",2,"[21.75, 21.75]"
5,"[CTNNB1, EGFR]","[11, 13]",2,"[27.51, 35.40]"
6,[HRAS],[28],1,[46.17]
7,"[KIT, MYC]","[34, 40]",2,"[53.12, 60.64]"


In [223]:
Tabla4Patologicas = pd.DataFrame({'Genes patológicos': patologicos, 'Número de la mutación específica':numero_iden_pato, '% frecuencia alélica':frecuenciasPato, 'Número de mutaciones patológicas': num_mutacionesPato})

In [224]:
Tabla4Patologicas

,Genes patológicos,Número de la mutación específica,% frecuencia alélica,Número de mutaciones patológicas
0,[],[],[],0
1,[KRAS],[35],[66.50],1
2,[],[],[],0
3,[],[],[],0
4,[],[],[],0
5,[CTNNB1],[11],[27.51],1
6,[],[],[],0
7,[],[],[],0


In [231]:
Tabla5Informacion = pd.DataFrame({'Ensayos clínicos': lista_ensayos,'SI/NO ensayo':ensayos_finales, 'Fármaco aprobado': lista_tratamientos, 'SI/NO fármacos': tratamientos_finales})

In [232]:
Tabla5Informacion

,Ensayos clínicos,SI/NO ensayo,Fármaco aprobado,SI/NO fármacos
0,6,1,6,1
1,5,1,1,1
2,0,0,0,0
3,0,0,0,0
4,3,1,3,1
5,5,1,0,0
6,1,1,0,0
7,0,0,1,1


In [233]:
tabla_12 = pd.concat([Tabla1Pacientes, Tabla2Diagnostico], axis=1)
tabla_12

,Número de chip,Número de paciente,NHC,Número de biopsia,Biopsia sólida,Fecha de informe,Diagnóstico,Número del diagnóstico
0,100,1,00021,23B00021-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1
1,100,2,00022,22B00022-A2,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1
2,100,3,00023,21B00023-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1
3,100,4,00024,23B00024-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1
4,100,5,00025,23C00025-A1,3,24-mar-2023,Carcinoma pulmonar no microcítico,15.1
5,100,6,00026,23P00026,2,24-mar-2023,Carcinoma pulmonar no microcítico,15.1
6,100,7,00027,22B00027-C4,1,24-mar-2023,Cáncer tiroideo,30.0
7,100,8,00028,23B00028-A2,1,24-mar-2023,Cáncer gástrico,11.0


In [234]:
tabla_123 = pd.concat([tabla_12, Tabla3MutacionesEspecificas], axis=1)
tabla_123

,Número de chip,Número de paciente,NHC,Número de biopsia,Biopsia sólida,Fecha de informe,Diagnóstico,Número del diagnóstico,Mutaciones detectadas,Número de la mutación específica,Total del número de mutaciones,Porcentaje de frecuencia alélica (ADN)
0,100,1,00021,23B00021-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[MYC],[40],1,[49.56]
1,100,2,00022,22B00022-A2,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[KRAS],[35],1,[66.50]
2,100,3,00023,21B00023-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],0,[]
3,100,4,00024,23B00024-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,"[FGFR1, PIK3CA]","[22, 47]",2,[]
4,100,5,00025,23C00025-A1,3,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,"[CDK6, MET]","[10, 38]",2,"[21.75, 21.75]"
5,100,6,00026,23P00026,2,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,"[CTNNB1, EGFR]","[11, 13]",2,"[27.51, 35.40]"
6,100,7,00027,22B00027-C4,1,24-mar-2023,Cáncer tiroideo,30.0,[HRAS],[28],1,[46.17]
7,100,8,00028,23B00028-A2,1,24-mar-2023,Cáncer gástrico,11.0,"[KIT, MYC]","[34, 40]",2,"[53.12, 60.64]"


In [235]:
tabla_final = pd.concat([tabla_123, Tabla5Informacion], axis=1)
tabla_final

,Número de chip,Número de paciente,NHC,Número de biopsia,Biopsia sólida,Fecha de informe,Diagnóstico,Número del diagnóstico,Mutaciones detectadas,Número de la mutación específica,Total del número de mutaciones,Porcentaje de frecuencia alélica (ADN),Ensayos clínicos,SI/NO ensayo,Fármaco aprobado,SI/NO fármacos
0,100,1,00021,23B00021-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[MYC],[40],1,[49.56],6,1,6,1
1,100,2,00022,22B00022-A2,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[KRAS],[35],1,[66.50],5,1,1,1
2,100,3,00023,21B00023-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],0,[],0,0,0,0
3,100,4,00024,23B00024-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,"[FGFR1, PIK3CA]","[22, 47]",2,[],0,0,0,0
4,100,5,00025,23C00025-A1,3,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,"[CDK6, MET]","[10, 38]",2,"[21.75, 21.75]",3,1,3,1
5,100,6,00026,23P00026,2,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,"[CTNNB1, EGFR]","[11, 13]",2,"[27.51, 35.40]",5,1,0,0
6,100,7,00027,22B00027-C4,1,24-mar-2023,Cáncer tiroideo,30.0,[HRAS],[28],1,[46.17],1,1,0,0
7,100,8,00028,23B00028-A2,1,24-mar-2023,Cáncer gástrico,11.0,"[KIT, MYC]","[34, 40]",2,"[53.12, 60.64]",0,0,1,1


In [236]:
tabla_124 = pd.concat([tabla_12, Tabla4Patologicas], axis=1)
tabla_124

,Número de chip,Número de paciente,NHC,Número de biopsia,Biopsia sólida,Fecha de informe,Diagnóstico,Número del diagnóstico,Genes patológicos,Número de la mutación específica,% frecuencia alélica,Número de mutaciones patológicas
0,100,1,00021,23B00021-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0
1,100,2,00022,22B00022-A2,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[KRAS],[35],[66.50],1
2,100,3,00023,21B00023-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0
3,100,4,00024,23B00024-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0
4,100,5,00025,23C00025-A1,3,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0
5,100,6,00026,23P00026,2,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[CTNNB1],[11],[27.51],1
6,100,7,00027,22B00027-C4,1,24-mar-2023,Cáncer tiroideo,30.0,[],[],[],0
7,100,8,00028,23B00028-A2,1,24-mar-2023,Cáncer gástrico,11.0,[],[],[],0


In [238]:
tabla_final_pato = pd.concat([tabla_124, Tabla4Patologicas], axis = 1)
tabla_final_pato

,Número de chip,Número de paciente,NHC,Número de biopsia,Biopsia sólida,Fecha de informe,Diagnóstico,Número del diagnóstico,Genes patológicos,Número de la mutación específica,% frecuencia alélica,Número de mutaciones patológicas,Genes patológicos,Número de la mutación específica,% frecuencia alélica,Número de mutaciones patológicas
0,100,1,00021,23B00021-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0,[],[],[],0
1,100,2,00022,22B00022-A2,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[KRAS],[35],[66.50],1,[KRAS],[35],[66.50],1
2,100,3,00023,21B00023-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0,[],[],[],0
3,100,4,00024,23B00024-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0,[],[],[],0
4,100,5,00025,23C00025-A1,3,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[],[],[],0,[],[],[],0
5,100,6,00026,23P00026,2,24-mar-2023,Carcinoma pulmonar no microcítico,15.1,[CTNNB1],[11],[27.51],1,[CTNNB1],[11],[27.51],1
6,100,7,00027,22B00027-C4,1,24-mar-2023,Cáncer tiroideo,30.0,[],[],[],0,[],[],[],0
7,100,8,00028,23B00028-A2,1,24-mar-2023,Cáncer gástrico,11.0,[],[],[],0,[],[],[],0
